# Figures for the Plos CB paper that use data from evolutionary runs

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import pathlib
import pickle
from numba import jit

In [ ]:
import importlib
import evotsc
import evotsc_lib
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_lib)
importlib.reload(evotsc_plot)

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/pci/main/')
gen=1_000_000

In [ ]:
rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
params = evotsc_lib.read_params(rep_dirs[0])
params['m'] = 2.5 # Temporary fix because the parameter wasn't saved

In [ ]:
params

In [ ]:
rng = np.random.default_rng(seed=123456)

In [ ]:
gene_types = ['AB', 'A', 'B'] # Name of each gene type
gene_type_color = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
dpi = 300

## Evolved individual: influence of env. supercoiling on final gene expression levels

In [ ]:
def compute_activity_sigma_per_type(indiv, sigmas):
    
    # Initialize the individual
    indiv.evaluate(0.0, 0.0)

    activ = np.zeros((3, len(sigmas))) # Compute activity for each gene type

    for i_sigma, sigma_env in enumerate(sigmas):
        # Evaluate the individual in the environment with sigma
        temporal_expr = indiv.run_system(sigma_env)

        # Compute total gene activation levels        
        for i_gene, gene in enumerate(indiv.genes):
            activ[gene.gene_type][i_sigma] += temporal_expr[-1, i_gene]
            
    activ /= (indiv.nb_genes / 3)
    
    return activ

In [ ]:
# See how gene activity levels depend on environmental supercoiling
def plot_activity_sigma_per_type(activ, sigmas, plot_title=None, plot_name=None):
    
    colors = ['tab:blue', 'tab:red', 'tab:green'] # AB: blue, A: red, B: green
            
        
    plt.figure(figsize=(6, 4), dpi=dpi)
    plt.xlabel('Environment supercoiling $\sigma_{env}$')
    plt.ylabel('Average gene expression by type')
    plt.ylim(-0.05, 1.05)
    plt.xlim(sigmas[0], sigmas[-1])
    plt.grid(linestyle=':')
    
    if plot_title:
        plt.title(plot_title)
        
    # Add average expression per gene type
    for i_gene_type, gene_type in enumerate(gene_types):
        plt.plot(sigmas, activ[i_gene_type, :],
                 color=gene_type_color[i_gene_type],
                 linewidth=2,
                 label=gene_type)

        
    # Add sigma_A and sigma_B
    y_min, y_max = plt.ylim()
    plt.vlines(params['sigma_A'], y_min, y_max, linestyle='--', linewidth=1, color='black')
    plt.vlines(params['sigma_B'], y_min, y_max, linestyle='--', linewidth=1, color='black')
    # To make math bold, use \mathbf{}
    plt.text(params['sigma_A'], y_min, '$\sigma_A$', va='top', fontsize='large')
    plt.text(params['sigma_B'], y_min, '$\sigma_B$', va='top', fontsize='large')
    plt.ylim(y_min, y_max)
    
    # Add 1/2 expression level
    half_expr = (1 + np.exp(- params['m'])) / 2
    plt.hlines(half_expr, sigmas[0], sigmas[-1], linestyle=':', linewidth=1,
           color='tab:pink')#, label='Activation threshold')
    
    
        # Add expression for an isolated gene
    activities = 1.0 / (1.0 + np.exp((sigmas + params['sigma_basal'] - params['sigma_opt']) / params['epsilon']))
    plt.plot(sigmas, np.exp(params['m'] * (activities - 1)), linewidth=2, color='tab:cyan', zorder=0,
             linestyle='--', label='Theory')
    
    plt.legend(loc='lower left')
    
    plt.tight_layout()
        
    if plot_name:
        plt.savefig(plot_name, dpi=dpi, bbox_inches='tight')
        
    plt.show()
    plt.close()

In [ ]:
def plot_best_activ_by_sigma():
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    nb_reps = len(rep_dirs)
    
    nb_sigmas = 250
    sigmas = np.linspace(-0.05, 0.05, nb_sigmas)

    for i_rep in range(nb_reps):
        indiv = evotsc_lib.get_best_indiv(exp_path.joinpath(f'rep{i_rep:02}'), gen=gen)
            
            
        activ = compute_activity_sigma_per_type(indiv, sigmas)

        plot_activity_sigma_per_type(activ, sigmas, plot_title=f'Best replicate {i_rep}',
                                     plot_name=exp_path.joinpath(f'activity_sigmas_best_rep{i_rep}.pdf'))

In [ ]:
#plot_best_activ_by_sigma()

In [ ]:
def plot_avg_best_activ_by_sigma():
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    nb_reps = len(rep_dirs)
    
    nb_sigmas = 250
    sigmas = np.linspace(-0.05, 0.05, nb_sigmas)
    activ = np.zeros((3, len(sigmas)))

    for i_rep in range(nb_reps):
        indiv = evotsc_lib.get_best_indiv(exp_path.joinpath(f'rep{i_rep:02}'), gen=gen)
            
            
        activ += compute_activity_sigma_per_type(indiv, sigmas)
        
    activ /= nb_reps

    plot_activity_sigma_per_type(activ, sigmas, #plot_title='Average over all replicas',
                                 plot_name=exp_path.joinpath(f'activity_sigmas_avg.pdf'))

In [ ]:
plot_avg_best_activ_by_sigma()

## See how SC and gene activity locally change after reversing or knocking out each gene

In [ ]:
@jit(nopython=True)
def run_system_numba_ko(nb_genes: int,
                  init_expr: np.ndarray,
                  inter_matrix: np.ndarray,
                  sigma_basal: float,
                  sigma_opt: float,
                  epsilon: float,
                  m: float,
                  sigma_env: float,
                  id_ko:int) -> np.ndarray:

    step_size = 0.5
    stop_dist = 1e-7
    max_eval_steps = 200

    temporal_expr = np.zeros((max_eval_steps+1, nb_genes))

    # Initial values at t = 0
    temporal_expr[0, :] = init_expr

    # Iterate the system
    it = 1
    cont = True
    while cont:
        prev_expr = temporal_expr[it-1, :]
        sigma_local = inter_matrix @ prev_expr
        sigma_total = sigma_basal + sigma_local + sigma_env

        promoter_activity = 1.0 / (1.0 + np.exp((sigma_total - sigma_opt)/epsilon))

        # We subtract 1 to rescale between exp(-m) and 1
        iter_expr = np.exp(m * (promoter_activity - 1.0))

        nouv_expr = step_size * iter_expr + (1 - step_size) * prev_expr
        
        # Knockout
        nouv_expr[id_ko] = 0

        temporal_expr[it, :] = nouv_expr

        # Check if we're done
        dist = np.abs(nouv_expr - prev_expr).sum() / nb_genes

        prev_expr = nouv_expr

        if dist < stop_dist:
            cont = False

        if it == max_eval_steps:
            cont = False
        it += 1

    temporal_expr = temporal_expr[:it, :]

    return temporal_expr

    
def run_system_ko(self, sigma_env, id_ko):

    init_expr = np.array([gene.basal_expression for gene in self.genes])
    init_expr[id_ko] = 0.0

    self.inter_matrix = self.compute_inter_matrix()

    return run_system_numba_ko(nb_genes=self.nb_genes,
                               init_expr=init_expr,
                               inter_matrix=self.inter_matrix,
                               sigma_basal=self.sigma_basal,
                               sigma_opt=self.sigma_opt,
                               epsilon=self.epsilon,
                               m=self.m,
                               sigma_env=sigma_env,
                               id_ko=id_ko)

In [ ]:
def count_genes_affected_by_change(indiv, change_type, sigma):
        
    if indiv.inter_matrix is None:
        indiv.inter_matrix = indiv.compute_inter_matrix()
    orig_activ = indiv.run_system(sigma)[-1, :] > (1 + np.exp(- indiv.m)) / 2

    total_diff = [0, 0, 0] # Count per gene type

    for i_changed in range(indiv.nb_genes):
        clone = indiv.clone()
        if change_type == 'inversion':
            clone.genes[i_changed].orientation = 1 - clone.genes[i_changed].orientation
            clone.inter_matrix = clone.compute_inter_matrix()
            clone_activ = clone.run_system(sigma)[-1, :] > (1 + np.exp(- indiv.m)) / 2
            
        elif change_type == 'knockout':
            clone_expr = run_system_ko(clone, sigma, id_ko=i_changed)
            clone_activ = clone_expr[-1, :] > (1 + np.exp(- indiv.m)) / 2

        for i_gene in range(indiv.nb_genes):
            if (i_gene != i_changed) and (orig_activ[i_gene] != clone_activ[i_gene]):
                # count by class of the modified gene
                total_diff[indiv.genes[i_changed].gene_type] += 1
        

    return total_diff

In [ ]:
def count_genes_affected_by_change_all(exp_path, gen, change_type, sigma):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    nb_reps = len(rep_dirs)
    
    count = [] # list of (1, 3) np arrays
    
    for i_rep in range(nb_reps):
        indiv = evotsc_lib.get_best_indiv(exp_path.joinpath(f'rep{i_rep:02}'), gen=gen)
        count.append(count_genes_affected_by_change(indiv, change_type, sigma))
        
    nb_genes_per_type = params["nb_genes"] / len(gene_types)
        
    return pd.DataFrame(count, columns=gene_types) / nb_genes_per_type

In [ ]:
def count_genes_affected_by_change_random(rand_indivs, change_type, sigma):
    count = []
        
    for indiv in rand_indivs:
        count.append(count_genes_affected_by_change(indiv, change_type, sigma))
        
    nb_genes_per_type = params["nb_genes"] / len(gene_types)
    
    return pd.DataFrame(count, columns=gene_types) / nb_genes_per_type

In [ ]:
def gather_genes_affected_by_change(exp_path, gen, change_type, nb_rand):
    
    # Experimental data
    exp_env_A = count_genes_affected_by_change_all(exp_path, gen, change_type, sigma=params['sigma_A'])
    exp_env_B = count_genes_affected_by_change_all(exp_path, gen, change_type, sigma=params['sigma_B'])

    # Random individuals
    rand_indivs = []
    mutation = evotsc.Mutation(inversion_poisson_lam=params['inversion_poisson_lam'])
    for i_rand in range(nb_rand):
        indiv = evotsc_lib.make_random_indiv(intergene=int(params['intergene']),
                                             gene_length=int(params['gene_length']),
                                             nb_genes=int(params['nb_genes']),
                                             default_basal_expression=params['default_basal_expression'],
                                             interaction_dist=params['interaction_dist'],
                                             interaction_coef=params['interaction_coef'],
                                             sigma_basal=params['sigma_basal'],
                                             sigma_opt=params['sigma_opt'],
                                             epsilon=params['epsilon'],
                                             m=params['m'],
                                             selection_coef=params['selection_coef'],
                                             mutation=mutation,
                                             rng=rng,
                                             nb_mutations=100)
        rand_indivs.append(indiv)
    
    rand_env_A = count_genes_affected_by_change_random(rand_indivs, change_type, sigma=params['sigma_A'])
    rand_env_B = count_genes_affected_by_change_random(rand_indivs, change_type, sigma=params['sigma_B'])
    
    return [exp_env_A, rand_env_A, exp_env_B, rand_env_B]

In [ ]:
affected_genes_ko = gather_genes_affected_by_change(exp_path, gen, 'knockout', nb_rand=100)

In [ ]:
affected_genes_inversion = gather_genes_affected_by_change(exp_path, gen, 'inversion', nb_rand=100)

In [ ]:
def plot_single_change_effect(affected_genes, change_type):
    
    fig, ax = plt.subplots(figsize=(8, 5), dpi=300)
        
    [exp_env_A, rand_env_A, exp_env_B, rand_env_B] = affected_genes
    
    width = 0.2
    x_pos = np.array([0, 4*width, 9*width, 13*width])
    delta = np.array([-width, 0, width])

    rects = {}
    
    for i_gene_type, gene_type in enumerate(gene_types):
        rects[gene_type] = plt.bar(x_pos + delta[i_gene_type],
                                   [exp_env_A[gene_type].mean(), rand_env_A[gene_type].mean(),
                                    exp_env_B[gene_type].mean(), rand_env_B[gene_type].mean()],
                                    width=width, color=gene_type_color[i_gene_type])
        plt.boxplot([exp[gene_type] for exp in affected_genes], positions=x_pos + delta[i_gene_type], 
                    manage_ticks=False, widths=0.1, medianprops={'color':'black'})
    
    for i_rect, rect in enumerate(rects['A']):  # middle rects
        plt.annotate(f"n = {len(affected_genes[i_rect]['A'])}",
                    xy=(rect.get_x() + rect.get_width()/2, 0),
                    xytext=(0, 3),
                    ha='center',
                    textcoords="offset points",
                    color='black')
    
    plt.xticks(ticks=x_pos, labels=['Evolved env. A', 'Random env. A', 'Evolved env. B', 'Random env. B'])

    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.ylabel(f'Genes switched on or off ({change_type})')
    plt.ylim(0, 11)
    plt.grid(axis='y', linestyle=':')
    
    patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
                    for color, label in zip(gene_type_color, gene_types)])
    plt.legend(handles=patches, title='Gene type')#, title_fontsize=15, fontsize=15)

    #plt.suptitle(exp_path.name)
    
    plt.savefig(exp_path.joinpath(f'switches_{change_type}.pdf'), bbox_inches='tight')
    plt.show()

In [ ]:
plot_single_change_effect(affected_genes_ko, change_type='knockout')

In [ ]:
plot_single_change_effect(affected_genes_inversion, change_type='inversion')